# 0 app store crawling

In [6]:
import os
os.getcwd()

'C:\\Users\\LG\\zepeto'

In [7]:
# 핑크색 warning 안내문 방지 
import warnings
warnings.filterwarnings('ignore')

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 1 크롤링에 필요한 라이브러리를 불러옵니다

In [9]:
import chromedriver_autoinstaller
import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import csv

# 2 국내 유저 리뷰 크롤링 

####  apple appstore는 크롤링을 할 수 있는 api를 제공하고 있다 
https://www.apptweak.io/documentation/ios/application_reviews_filtered

#### 제페토 apple appstore id
https://apps.apple.com/app/id1350301428
#### 제페토 googelp play id
https://play.google.com/store/apps/details?id=me.zepeto.main

## 2.1 RSS를 이용하여 앱스토어 리뷰 크롤링 (XML 형식)

In [10]:
# 모듈 로딩

import pandas as pd #크롤링한 데이터를 데이터프레임으로 담아줄 것

import xmltodict #xml 파싱을 위한 모듈
from urllib.request import urlopen # url을 이용하여 웹상의 데이터를 가져올 irllib.request
import json #xmltodict의 결과물을 dict로 바꾸기 위함 

### 2.1.1 Get Data from URL (page 1 파싱)

In [ ]:
# 하단 url로 접속해서 xml데이터를 파싱할 것임 
#reopen 함수를 사용하여 해당urk이 가지는 값을 가져온다
url = 'https://itunes.apple.com/kr/rss/customerreviews/page=1/id=1350301428/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1350301428/sortBy=mostRecent/xml'
response = urlopen(url).read()
response

### 2.1.2 Parsing XML

In [ ]:
xml = xmltodict.parse(response)
XmlToJson = json.loads(json.dumps(xml))

### 2.1.3 파싱한 xml 데이터를 내가 원하는대로 바꾸기

In [ ]:
res_dict = []

for i in range(len(XmlToJson['feed']['entry'])):
    res_dict.append({
        'DATE' : XmlToJson['feed']['entry'][i]['updated'],
        'STAR' : XmlToJson['feed']['entry'][i]['im:rating'],
        'LIKE' : XmlToJson['feed']['entry'][i]['im:voteSum'],
        'DISLIKE' : int(XmlToJson['feed']['entry'][i]['im:voteCount']) - int(XmlToJson['feed']['entry'][i]['im:voteSum']),
        'TITLE' : XmlToJson['feed']['entry'][i]['title'],
        'REVIEW' : XmlToJson['feed']['entry'][i]['content'][0]['#text']
    })

In [ ]:
# 잘 변환 되었는지 확인
res_dict

In [ ]:
#ㅔpage 1 파싱 내용을 데이터프레임으로 담아줍니다 
res_df = pd.DataFrame(res_dict)
res_df

In [ ]:
#시계열 컬럼을 예쁘게 수정해주기
res_df['DATE'] = pd.to_datetime(res_df['DATE'], format="%Y-%m-%dT%H:%M:%S-07:00")
res_df.head()

### 2.1.4 반복문으로 page 1~ 10 까지 한번에 긁어오기

In [ ]:
res_dict = []
for j in range(1,11):
    url = 'https://itunes.apple.com/kr/rss/customerreviews/page=%i/id=1350301428/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1350301428/sortBy=mostRecent/xml' % j
    response = urlopen(url).read()

    xml = xmltodict.parse(response)
    XmlToJson = json.loads(json.dumps(xml))

    for i in range(len(XmlToJson['feed']['entry'])):
        res_dict.append({
            'DATE' : XmlToJson['feed']['entry'][i]['updated'],
            'STAR' : int(XmlToJson['feed']['entry'][i]['im:rating']),
            'LIKE' : int(XmlToJson['feed']['entry'][i]['im:voteSum']),
            'DISLIKE' : int(XmlToJson['feed']['entry'][i]['im:voteCount']) - int(XmlToJson['feed']['entry'][i]['im:voteSum']),
            'TITLE' : XmlToJson['feed']['entry'][i]['title'],
            'REVIEW' : XmlToJson['feed']['entry'][i]['content'][0]['#text']
        })

res_df = pd.DataFrame(res_dict)
res_df['DATE'] = pd.to_datetime(res_df['DATE'], format="%Y-%m-%dT%H:%M:%S-07:00")
#csv로 저장
res_df.to_csv('data/zepeto_kr_review.csv', index = False, encoding = 'utf-8')

In [ ]:
#잘 저장 되었는지 확인하기
df = pd.read_csv('data/zepeto_kr_review.csv', encoding = 'utf-8')
df.head()

In [ ]:
df['DATE'].sort_values(ascending = True)

# 3 해외 유저 리뷰 크롤링

## 3.1 미국(US) 서버 크롤링

In [ ]:
res_dict = []
for j in range(1,11):
    url = 'https://itunes.apple.com/us/rss/customerreviews/page=%i/id=1350301428/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1350301428/sortBy=mostRecent/xml' % j
    response = urlopen(url).read()

    xml = xmltodict.parse(response)
    XmlToJson = json.loads(json.dumps(xml))

    for i in range(len(XmlToJson['feed']['entry'])):
        res_dict.append({
            'DATE' : XmlToJson['feed']['entry'][i]['updated'],
            'STAR' : int(XmlToJson['feed']['entry'][i]['im:rating']),
            'LIKE' : int(XmlToJson['feed']['entry'][i]['im:voteSum']),
            'DISLIKE' : int(XmlToJson['feed']['entry'][i]['im:voteCount']) - int(XmlToJson['feed']['entry'][i]['im:voteSum']),
            'TITLE' : XmlToJson['feed']['entry'][i]['title'],
            'REVIEW' : XmlToJson['feed']['entry'][i]['content'][0]['#text']
        })

res_df = pd.DataFrame(res_dict)
res_df['DATE'] = pd.to_datetime(res_df['DATE'], format="%Y-%m-%dT%H:%M:%S-07:00")
#csv로 저장
res_df.to_csv('data/zepeto_us_review.csv', index = False, encoding = 'utf-8')

In [ ]:
#잘 저장 되었는지 확인하기
df = pd.read_csv('data/zepeto_us_review.csv', encoding = 'utf-8')
df.head()

## 3.2 일본(JP) 서버 크롤링

In [ ]:
res_dict = []
for j in range(1,11):
    url = 'https://itunes.apple.com/jp/rss/customerreviews/page=%i/id=1350301428/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1350301428/sortBy=mostRecent/xml' % j
    response = urlopen(url).read()

    xml = xmltodict.parse(response)
    XmlToJson = json.loads(json.dumps(xml))

    for i in range(len(XmlToJson['feed']['entry'])):
        res_dict.append({
            'DATE' : XmlToJson['feed']['entry'][i]['updated'],
            'STAR' : int(XmlToJson['feed']['entry'][i]['im:rating']),
            'LIKE' : int(XmlToJson['feed']['entry'][i]['im:voteSum']),
            'DISLIKE' : int(XmlToJson['feed']['entry'][i]['im:voteCount']) - int(XmlToJson['feed']['entry'][i]['im:voteSum']),
            'TITLE' : XmlToJson['feed']['entry'][i]['title'],
            'REVIEW' : XmlToJson['feed']['entry'][i]['content'][0]['#text']
        })

res_df = pd.DataFrame(res_dict)
res_df['DATE'] = pd.to_datetime(res_df['DATE'], format="%Y-%m-%dT%H:%M:%S-07:00")
#csv로 저장
res_df.to_csv('data/zepeto_jp_review.csv', index = False, encoding = 'utf-8')

In [ ]:
#잘 저장 되었는지 확인하기
df = pd.read_csv('data/zepeto_jp_review.csv', encoding = 'utf-8')
df.head()

## 3.3 중국(cn) 서버 크롤링  -> 막힘 ㅠ 

In [11]:
# 하단 url로 접속해서 xml데이터를 파싱할 것임 
#reopen 함수를 사용하여 해당urk이 가지는 값을 가져온다
url = 'https://itunes.apple.com/cn/rss/customerreviews/page=1/id=1350301428/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1350301428/sortBy=mostRecent/xml'
response = urlopen(url).read()
response

b'<?xml version="1.0" encoding="utf-8"?>\n\n\t<feed xmlns:im="http://itunes.apple.com/rss" xmlns="http://www.w3.org/2005/Atom" xml:lang="zh">\n\t\t<id>https://mzstoreservices-int-st.itunes.apple.com/cn/rss/customerreviews/page=1/id=1350301428/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1350301428/sortBy=mostRecent/xml</id><title>iTunes Store\xef\xbc\x9a\xe7\x94\xa8\xe6\x88\xb7\xe8\xaf\x84\xe8\xae\xba</title><updated>2022-07-24T02:15:18-07:00</updated><link rel="alternate" type="text/html" href="https://apps.apple.com/WebObjects/MZStore.woa/wa/viewGrouping?cc=cn&amp;id=29099"/><link rel="self" href="https://mzstoreservices-int-st.itunes.apple.com/cn/rss/customerreviews/page=1/id=1350301428/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1350301428/sortBy=mostRecent/xml"/><link rel="first" href=""/><link rel="last" href=""/><link rel="previous" href=""/><link rel="next" href=""/><icon>http://itunes.apple.com/favicon.ico</icon><author><name>iTunes Store</name><uri>http://www.apple

In [ ]:
res_dict = []
for j in range(1,11):
    url = 'https://itunes.apple.com/cn/rss/customerreviews/page=%i/id=1350301428/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1350301428/sortBy=mostRecent/xml' % j
    response = urlopen(url).read()

    xml = xmltodict.parse(response)
    XmlToJson = json.loads(json.dumps(xml))

    for i in range(len(XmlToJson['feed']['entry'])):
        res_dict.append({
            'DATE' : XmlToJson['feed']['entry'][i]['updated'],
            'STAR' : int(XmlToJson['feed']['entry'][i]['im:rating']),
            'LIKE' : int(XmlToJson['feed']['entry'][i]['im:voteSum']),
            'DISLIKE' : int(XmlToJson['feed']['entry'][i]['im:voteCount']) - int(XmlToJson['feed']['entry'][i]['im:voteSum']),
            'TITLE' : XmlToJson['feed']['entry'][i]['title'],
            'REVIEW' : XmlToJson['feed']['entry'][i]['content'][0]['#text']
        })

res_df = pd.DataFrame(res_dict)
res_df['DATE'] = pd.to_datetime(res_df['DATE'], format="%Y-%m-%dT%H:%M:%S-07:00")
#csv로 저장
res_df.to_csv('data/zepeto_jp_review.csv', index = False, encoding = 'utf-8')